## 1. Steam Dataset: Data Engineering 
Website: https://steam.internet.byu.edu/

* A dataset collected and analyzed for the 2016 ACM Internet Measurement Conference article by Mark O'Neill, Justin Wu, Elham Vaziripour, and Daniel Zappala

In [3]:
import findspark
findspark.init("D:\Python\spark-3.0.0-preview2-bin-hadoop2.7")

from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
sc = SparkContext('local[*]')  # local cluster (cpu cores)
sql_sc = SQLContext(sc)

### 2.	Load `Player_Summaries`, `Game_Publishers`, `Game_Genres`, `Game_Developers`, `Games_1` into PySpark dataframes 

In [5]:
%%time
pla = sql_sc.read.options(header=True, inferSchema=True) \
            .csv('data/small/Player_Summaries.csv')

pub = sql_sc.read.options(header=True, inferSchema=True) \
            .csv('data/small/Games_Publishers.csv')

gen = sql_sc.read.options(header=True, inferSchema=True) \
            .csv('data/small/Games_Genres.csv')

dev = sql_sc.read.options(header=True, inferSchema=True) \
            .csv('data/small/Games_Developers.csv')

ga1 = sql_sc.read.options(header=True, inferSchema=True) \
            .csv('data/small/Games_1.csv')

ga2 = sql_sc.read.options(header=True, inferSchema=True) \
            .csv('data/small/Games_2.csv')

app = sql_sc.read.options(header=True, inferSchema=True) \
            .csv('data/small/App_ID_Info.csv')


Wall time: 14 s


### 3.	Join all `Games_` tables into one dataframe

* Data lists each game played per user.
* The two tables have the same data and same users, the second `ga2` contains the game data from a follow-up (larger) crawl of the Steam network.
* Duplicate user/game data should be filtered before aggregation


##### UNION: `GAMES_1` and `GAMES_2` 

In [6]:
all_games = ga1.union(ga2)

print(f"{all_games.select('steamid').distinct().count():,} People")
print(f" {all_games.select('appid').distinct().count():,} Different games/apps")
print(f"\n{all_games.count():,} Total rows")

80,673 People
 3,975 Different games/apps

1,828,674 Total rows


In [19]:
ga1.count(), ga2.count()

(628117, 1200557)

In [16]:
all_games.groupBy('steamid', 'appid').count().count()

1200557

### 4.	Count the number of games per `publisher` and per `genre`.

* Games have multiple tagged `genres`
* Some games have multiple `publishers` (all counted as equal)

##### Top 10 Publishers by Game Count

In [51]:
pub_df = app.join(pub, on='appid') \
            .orderBy("Release_Date", ascending=False)

pub_df[pub_df.Type == 'game'].groupBy(['Publisher']).count() \
                             .orderBy('count', ascending=False) \
                             .limit(10).toPandas()

,Publisher,count
0,SEGA,128
1,KISS ltd,124
2,Strategy First,123
3,Ubisoft,115
4,Nordic Games,96
5,Night Dive Studios,93
6,Square Enix,75
7,Disney Interactive,70
8,1C Company,69
9,Black Shell Media,66


##### Top 10 Genres by Game Count

In [52]:
gen_df = app.join(gen, on='appid') \
            .orderBy("Release_Date", ascending=False)

gen_df[gen_df.Type == 'game'].groupBy(['Genre']).count() \
                             .orderBy('count', ascending=False) \
                             .limit(10).toPandas()

,Genre,count
0,Indie,5317
1,Action,4009
2,Adventure,2981
3,Casual,2237
4,Strategy,1998
5,RPG,1588
6,Simulation,1418
7,Early Access,891
8,Free to Play,450
9,Racing,352


### 5.	Find day and hour when most new accounts were `created`.

(based on `Player_Summaries` table) e.g. 8pm on 14th August 2005.


In [53]:
from pyspark.sql import functions as F

df = pla.withColumn('dt_created',
                    F.to_date(F.unix_timestamp('timecreated', 'yyyy-MM-dd HH:mm:ss')
                     .cast('timestamp')))

df = df.withColumn("dt_hour", F.hour(F.col("timecreated")))

In [55]:
%%time

dt = df.where(F.col("dt_created").isNotNull()) \
       .groupBy(['dt_created', 'dt_hour']).count() \
       .orderBy('count', ascending=False).toPandas()

dt.head(10)

Wall time: 17.5 s


,dt_created,dt_hour,count
0,2012-12-25,10,292
1,2012-12-25,8,279
2,2012-12-25,11,230
3,2012-11-28,11,219
4,2012-11-28,10,216
5,2012-11-28,9,215
6,2012-12-25,7,208
7,2012-12-25,4,208
8,2012-11-28,8,205
9,2012-12-24,11,204


* The most popular time for account creation was `10am Christmas Day, 2012` 🎁